<h2 align=center> Facial Expression Recognition with Keras</h2>

In [ ]:
import tarfile
fname = '../input/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.tar.gz'
if fname.endswith("tar.gz"):
    tar = tarfile.open(fname , "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

### Task 1: Import Libraries

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install utils
!pip install livelossplot


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import pandas as pd
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from keras.utils import np_utils

from IPython.display import SVG, Image
# from livelossplot import PlotLossesTensorFlowKeras
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

## append two csv file into one file for trainning

In [ ]:
df=pd.read_csv('./fer2013/fer2013.csv')
df2 = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv')
df.head(4)

In [ ]:
df.drop(['Usage'] , axis =1, inplace =True)

In [ ]:
df2.columns

In [ ]:
print("number of images is : " ,df.shape[0])

In [ ]:
print("number of images is : " ,df2.shape[0])

In [ ]:
df2.columns

In [ ]:
# number of labels 

df['emotion'].value_counts()

## summary about data 

1- number of images is 35887

2- number of label is 7 label from(0 - 6)


In [ ]:
label =[]
data  =[]

In [ ]:
## convert pixels into two categories
# read first file

import csv


flag = True

with open ("./fer2013/fer2013.csv", 'r') as f:
    
    reader = csv.reader (f, delimiter=',')
    for row in reader:
    # skip first row
        if flag == True:
            flag = False

        else:
            label.append(row[0])
            data.append([int(p) for p in row[1].split()])
 



In [ ]:
#read second file

import csv


flag = True

with open ("../input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv", 'r') as f:
    
    reader = csv.reader (f, delimiter=',')
    for row in reader:
    # skip first row
        if flag == True:
            flag = False

        else:
            label.append(row[0])
            data.append([int(p) for p in row[1].split()])
 

In [ ]:
len(data)

In [ ]:
data = np.array(data)

In [ ]:
label = np.array(label)

In [ ]:
## reshape image into 48*48
sample,w = data.shape

data = data.reshape(sample , 48 ,48 ,1)

In [ ]:
data.shape

### Task 2: Plot Sample Image

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=0)


In [ ]:
print("x_train is : " , X_train.shape)
print("y_train is : " , y_train.shape)
print("X_test is : " , X_test.shape)
print("y_test is : " , y_test.shape)

In [ ]:
## scale image 
X_train = X_train/255.0
X_test  = X_test/255.0


In [ ]:
X_test[0]

In [ ]:
class_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutr']

In [ ]:
# return class name
def class_name(label):
    return class_names[label]

In [ ]:
from skimage.color import rgb2gray


plt.figure(figsize = (20,10))
for i in range(20):
    plt.subplot(4,5,i+1)
    index = np.random.choice(X_train.shape[0] , 1 , replace =False)
    img = X_train[index].reshape(48,48)
    img = rgb2gray(img)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel(class_name(int(y_train[index])))

### Task 3: Generate Training and Validation Batches

In [ ]:
## hyperparameter 

width  = 48
height = 48

batch_size = 64



### Task 4: Create CNN Model

![](model.png)
Inspired by Goodfellow, I.J., et.al. (2013). Challenged in representation learning: A report of three machine learning contests. *Neural Networks*, 64, 59-63. [doi:10.1016/j.neunet.2014.09.005](https://arxiv.org/pdf/1307.0414.pdf)

In [ ]:
model = Sequential()

#first convd
model.add(Conv2D(64 , 3 , activation = 'relu' , padding ='same' , input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))
    
#second convd
model.add(Conv2D(128 , 5 , activation = 'relu' , padding ='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))

#third convd
model.add(Conv2D(512 , 3 , activation = 'relu' , padding ='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))

#fourth convd
model.add(Conv2D(512 , 3 , activation = 'relu' , padding ='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.25))

#flatten

model.add(Flatten())

#first Dense
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

#second Dense
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

#output
model.add(Dense(7 , activation = 'softmax'))


model.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'adam' , metrics = ['acc']
        
    )
 
model.summary()    
    

In [ ]:
## convert y_train into one hot encoder
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(y_train)


In [ ]:
## convert y_train into one hot encoder
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y_test)
y_test = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y_test = np_utils.to_categorical(y_test)

In [ ]:
y_test[54]

In [ ]:
y_train[3333]

In [ ]:
y_train.shape

### Task 6: Train and Evaluate Model

In [ ]:
steps_per_epoch = X_train.shape[0]/batch_size
steps_per_epochs_validation = X_test.shape[0]/batch_size

steps_per_epochs_validation

In [ ]:
epochs = 15
steps_per_epoch = X_train.shape[0]/batch_size
steps_per_epochs_validation = X_test.shape[0]/batch_size

Checkpoint = ModelCheckpoint('model.h5', 
                        save_best_only=True ,
                        save_weights_only = False ,
                        mode = 'max',
                        monitor = 'val_accuracy')

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss' , 
                              factor = 0.1 , 
                              patience = 2 , 
                              min_lr = 0.00001 ,
                              mode = 'auto' )

 

callbacks = [Checkpoint , reduce_lr , EarlyStopping(monitor='val_accuracy')]



In [ ]:

history = model.fit(X_train,     
                    y_train, 
                    steps_per_epoch = steps_per_epoch,
                    epochs=25,
                    validation_data=(X_test,y_test),   
                    validation_steps = steps_per_epochs_validation,
                    callbacks=callbacks
                   
           
            )

In [ ]:
model.save('./fer2013/model.h5')

In [ ]:
## test model using test data

test = pd.read_csv('../input/challenges-in-representation-learning-facial-expression-recognition-challenge/test.csv')
test.shape

In [ ]:
with open ("../input/challenges-in-representation-learning-facial-expression-recognition-challenge/test.csv", 'r') as f:
    data1=[]
    flag = True
    reader = csv.reader (f, delimiter=',')
    for row in reader:
        
    # skip first row
        if flag == True:
            flag = False

        else:
            
            data1.append([int(p) for p in row[0].split()])
 

 

In [ ]:
data1 = np.array(data1)

sample,w = data1.shape

data1 = data1.reshape(sample , 48 ,48 ,1)




In [ ]:
data1 = data1/255.0

In [ ]:
test_pred = model.predict(data1)
np.argmax(test_pred[5])

### Task 7: Represent Model as JSON String

In [ ]:
model_json = model.to_json()

with open("model.json","w") as json_file:
    json_file.write(model_json)

In [ ]:
# green label is correct 
# red_label is not correct


# visualize some of image
plt.figure(figsize = (20,10))
for i in range(10):
    plt.subplot(2,5,i+1)
    index = np.random.choice(X_test.shape[0] , 1 , replace =False)
    img = X_test[index].reshape(48,48)
    img = rgb2gray(img)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

    if np.argmax(test_pred[index]) == np.argmax(y_test[index]):
        plt.xlabel(class_name(np.argmax(test_pred[index]))  , color = 'green')
    else:
        plt.xlabel(class_name(np.argmax(test_pred[index]))  , color = 'red')

In [ ]:
plt.figure(figsize = (15,15))
accs = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(range(len(accs)) , accs , label = 'trainning')
plt.plot(range(len(accs)) , val_acc , label = 'validation')
plt.legend()

In [ ]:
from sklearn import metrics

matrix = metrics.confusion_matrix(np.argmax(y_test , axis =1), np.argmax(test_pred , axis =1))
matrix

In [ ]:
np.argmax(data1[787])

In [ ]:
# visualization on test data


# visualize some of image
plt.figure(figsize = (20,10))
for i in range(10):
    plt.subplot(2,5,i+1)
    index = np.random.choice(data1.shape[0] , 1 , replace =False)
    img = data1[index].reshape(48,48)
    img = rgb2gray(img)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

    plt.xlabel(class_name(np.argmax(test_pred[index]))  , color = 'green')
    